In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/SST.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    seq_length = 0
    with open(filePath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = int(word[0])
            content = word[1:]
            train_data.append([content,label])
            
            if (len(content) > seq_length):
                seq_length = len(content)
    
    np.random.shuffle(train_data)
    return np.asarray(train_data), seq_length


def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i])  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [4]:
categories = ['very negative', 'negative', 'neutral', 'positive', 'very positive']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)

trainData, seq_length = readfile(trainDataPath)
print(len(trainData), seq_length)

# seq_length = 37

Loading GloVe!
Completed!
400000 100 400000
11748 56


In [5]:
def train_10_fold(train_data, categories):
    
    tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
    print(len(tx),len(tx[0]),len(tx[1]))
    
    fold_id = 0
    test_acc = []
    
    kf = KFold(n_splits=10)
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        test_acc.append(classifier.train(
            X_train=tx[train_i],
            y_train=ty[train_i],
            X_eval=tx[test_i],
            y_eval=ty[test_i],
            categories=categories,
            epochs=30
        ))
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class KimConvolutionalModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1408.5882.pdf
    '''
    def __init__(self,
        embeddings_configuration,
        conv_configurations = [(3, 100), (4, 100), (5, 100)],
        drop_rate           = 0.5):
        '''Constructor.
        # Parameters:
        embeddings: List of embeddings configuration. Each configuration is a
            pair of the form (embedding, trainable). `embedding` is a numpy
            array and `trainable` is a boolean that indicates whether that
            embedding is trainable or not.
        conv_configurations: List of pairs. Each pair represents a
            convolution configuration. Each configuration determines the
            size and number of each filter.
        '''

        self._embeddings_configuration = embeddings_configuration
        self._conv_configurations = conv_configurations
        self._drop_rate = drop_rate

    def __call__(self, input):
        self._embeddings_tf = tf.stack(
            values = [
                self._create_embedding_layer(e, input)
                for e in self._embeddings_configuration],
            axis = 1
        )

        self._convolutions_tf = self._create_convolutional_layers(
            self._conv_configurations, self._embeddings_tf)
        
        self._add_tf = self._create_add_layers(self._convolutions_tf)

        self._poolings_tf = self._create_maxpooling_layer(
            self._add_tf)

        self._reshape_tf = self._create_reshape_layer(self._poolings_tf)
        self._dropout_tf = tf.nn.dropout(
            self._reshape_tf,
            keep_prob = self._drop_rate)

        return self._dropout_tf

    def summary(self):
        print('embedding:', str(self._embeddings_tf.shape))
        for c in self._convolutions_tf:
            print('conv:', str(c.shape))
        for a in self._add_tf:
            print('add:', str(a.shape))
        for p in self._poolings_tf:
            print('pool:', str(p.shape))
        print('reshape:', str(self._reshape_tf.shape))

    def _create_embedding_layer(self, embedding_configuration, input_x):
        return tf.nn.embedding_lookup(
            params = tf.Variable(
                initial_value = embedding_configuration[0],
                trainable     = embedding_configuration[1]),
            ids = tf.cast(input_x, 'int32')
        )

    def _create_convolutional_layers(self, configuration, input_embedding):
        '''Creates the convolutional layers.
        # Parameters:
        configuration: A list. It must be of the form
            [(filter_size, num_filters), ...]
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the configuration 'i'.
        '''
        convolutions = []
        for filter_height, num_filters in configuration:
            filter_width = input_embedding.shape[3].value
            filter_shape = [1, filter_height, filter_width, num_filters]

            # Create weights and bias
            W = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=filter_shape,
                    stddev=0.1))
            b = tf.Variable(
                initial_value=tf.truncated_normal(
                    shape=[num_filters]))

            conv = tf.nn.conv2d(
                input=input_embedding,
                filter=W,
                strides=[1, 1, 1, 1],
                padding="VALID")
            bias = tf.nn.bias_add(conv, b)
            h = tf.nn.relu(bias)
            convolutions.append(h)

        return convolutions

    def _create_add_layers(self, convolutions):
        return [
            tf.reduce_sum(
                input_tensor = c,
                axis=1,
                keepdims=True)
            for c in convolutions
        ]

    def _create_maxpooling_layer(self, tensors):
        '''Creates the maxpooling layer. Computes maxpooling on each node
        # Parameters:
        input_convolutions: List of tensorflow nodes.
        # Returns:
        A list of tensorflow nodes. Each node 'i' computes the maxpooling of node 'i'
        '''
        return [
            tf.reshape(
                tensor = tf.nn.max_pool(
                    value=t,
                    ksize=[1, 1, t.shape[2], 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID'),
                shape = [-1, t.shape[3]]
            )
            for t in tensors
        ]

    def _create_reshape_layer(self, tensors):
        '''Creates a flatten layer
        '''
        return tf.concat(tensors, axis=1)

In [8]:
word_vector = embedding.astype('float32')
model = KimConvolutionalModel(
        embeddings_configuration=[
            (word_vector, True)
        ]
    )

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 56)
embedding: (?, 1, 56, 100)
conv: (?, 1, 54, 100)
conv: (?, 1, 53, 100)
conv: (?, 1, 52, 100)
add: (?, 1, 54, 100)
add: (?, 1, 53, 100)
add: (?, 1, 52, 100)
pool: (?, 100)
pool: (?, 100)
pool: (?, 100)
reshape: (?, 300)
output: (?, 5)


In [9]:
train_10_fold(trainData, categories)

11748 56 56
Fold:  1
Epoch: 1, Train Loss:  1.949, Train Acc: 27.64%, Val Loss:  1.503, Val Acc: 34.72%, Time: 44.75s *
Epoch: 2, Train Loss:  1.376, Train Acc: 40.27%, Val Loss:  1.419, Val Acc: 37.45%, Time: 43.37s *
Epoch: 3, Train Loss:  1.188, Train Acc: 49.80%, Val Loss:  1.413, Val Acc: 37.79%, Time: 44.47s *
Epoch: 4, Train Loss:  1.024, Train Acc: 58.08%, Val Loss:  1.405, Val Acc: 39.83%, Time: 44.77s *
Epoch: 5, Train Loss: 0.8555, Train Acc: 66.36%, Val Loss:  1.456, Val Acc: 41.28%, Time: 45.14s *
Epoch: 6, Train Loss:  0.704, Train Acc: 73.37%, Val Loss:  1.542, Val Acc: 39.57%, Time: 44.55s 
Epoch: 7, Train Loss: 0.5674, Train Acc: 79.26%, Val Loss:  1.696, Val Acc: 38.98%, Time: 44.43s 
Epoch: 8, Train Loss:  0.461, Train Acc: 83.48%, Val Loss:  1.841, Val Acc: 37.79%, Time: 44.39s 
Epoch: 9, Train Loss: 0.3789, Train Acc: 86.71%, Val Loss:  1.883, Val Acc: 40.00%, Time: 44.79s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
        

Epoch: 15, Train Loss: 0.1459, Train Acc: 95.22%, Val Loss:  2.825, Val Acc: 38.81%, Time: 44.80s 
Epoch: 16, Train Loss: 0.1348, Train Acc: 95.63%, Val Loss:  2.868, Val Acc: 38.30%, Time: 44.79s 
Epoch: 17, Train Loss:   0.12, Train Acc: 96.14%, Val Loss:  3.156, Val Acc: 40.43%, Time: 43.34s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
               precision    recall  f1-score   support

very negative       0.33      0.27      0.30       153
     negative       0.39      0.51      0.44       287
      neutral       0.29      0.25      0.27       219
     positive       0.46      0.47      0.47       312
very positive       0.54      0.45      0.49       204

    micro avg       0.41      0.41      0.41      1175
    macro avg       0.40      0.39      0.39      1175
 weighted avg       0.41      0.41      0.41      1175

Confusion Matrix...
[[ 41  81  15  11   5]
 [ 42 147  51  39   8]
 [ 27  79  54  54   5]
 [ 10  49  45 148  60]
 [  3  18

[0.3923404255319149,
 0.3719148936170213,
 0.4093617021276596,
 0.38127659574468087,
 0.4102127659574468,
 0.3880851063829787,
 0.4136170212765957,
 0.37446808510638296,
 0.36712095400340716,
 0.36541737649063033]